In [1]:
%cd /home/mbudisic/Documents/PsTuts-RAG
import os
from getpass import getpass
from dotenv import load_dotenv
from pstuts_rag.configuration import Configuration
import asyncio

import nest_asyncio
nest_asyncio.apply()


/home/mbudisic/Documents/PsTuts-RAG


In [2]:
import pstuts_rag

In [3]:
import logging
logging.basicConfig(
    level=Configuration().eva_log_level,
    format='%(asctime)s - %(levelname)s - %(funcName)s - %(message)s'
)

In [4]:
%load_ext autoreload
%autoreload 2


In [5]:
load_dotenv()


True

In [6]:
from pstuts_rag.rag_for_transcripts import *

2025-05-30 20:08:35,183 - INFO - <module> - Loaded .env file


In [7]:
Configuration().print(logging.info)

2025-05-30 20:08:36,978 - INFO - print - Configuration parameters:
2025-05-30 20:08:36,980 - INFO - print -   eva_workflow_name: EVA_workflow
2025-05-30 20:08:36,980 - INFO - print -   eva_log_level: INFO
2025-05-30 20:08:36,981 - INFO - print -   transcript_glob: ./data/dev.json:./data/test.json
2025-05-30 20:08:36,982 - INFO - print -   embedding_model: mxbai-embed-large
2025-05-30 20:08:36,983 - INFO - print -   embedding_api: ModelAPI.OLLAMA
2025-05-30 20:08:36,984 - INFO - print -   llm_api: ModelAPI.OLLAMA
2025-05-30 20:08:36,985 - INFO - print -   max_research_loops: 2
2025-05-30 20:08:36,986 - INFO - print -   llm_tool_model: deepseek-r1:8b
2025-05-30 20:08:36,987 - INFO - print -   n_context_docs: 3


In [8]:
datastore:DatastoreManager = startup(callback_on_loading_complete=lambda _: logging.warning("Loading complete.")) 

2025-05-30 20:08:37,093 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 20:08:37,118 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


In [9]:
await datastore.wait_for_loading()

2025-05-30 20:08:38,120 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 20:08:39,173 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 20:08:39,862 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 20:08:40,765 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 20:08:41,275 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 20:08:41,539 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 20:08:42,447 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 20:08:43,415 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 

True

In [10]:
chain = retrieve_videos(datastore)

In [11]:
response = chain.invoke({"question":"What is a layer?"})

2025-05-30 20:08:59,268 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-30 20:09:11,924 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [12]:
response.pretty_print()

================================== Ai Message ==================================

<think>
Okay, I need to figure out what a layer is based on the provided transcript. Let me go through the context step by step.

First, looking at the first document with video ID 19172. The title says "Understand layers" and the description mentions that layers are the building blocks of any image in Photoshop CC. So, layers must be fundamental components.

In the page content, it compares layers to separate flat prints of glass stacked on top of each other. Each layer has different content. That makes me think of layers as separate elements or parts of an image that can be edited individually.

There's also a mention of the Layers panel where you select and work with layers. It shows 4 layers, each with distinct content. The Eye icon can toggle visibility, so layers can be shown or hidden. This suggests that layers are like different layers of content that can be managed separately.

Looking at another

In [13]:
response.additional_kwargs["context"]

[Document(metadata={'video_id': 19172, 'title': 'Understand layers', 'desc': 'Learn what layers are and why they are so useful.', 'length': '00:04:44.75', 'group': 'data/test.json', 'source': 'https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4', 'speech_start_stop_times': [[0.47, 3.41], [3.81, 9.13], [9.309999, 15.01], [15.299999, 20.57], [20.88, 23.3], [23.83, 27.93], [29.38, 32.79], [32.96, 33.92], [34.43, 40.21], [41.91, 45.37], [45.88, 49.01], [49.54, 55.130001], [55.72, 58.49], [58.72, 62.14]], 'start': 0.47, 'stop': 62.14, '_id': 63, '_collection_name': 'dc0cf104-0069-4983-8a12-8d3de4132745'}, page_content="Layers are the building blocks of any image in Photoshop CC. So, it's important to understand, what layers are and why to use them - which we'll cover in this video. If you're following along, open this layered image from the downloadable practi